In [1]:
import requests
import json
import re
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
url = 'https://hh.ru'
route = '/search/vacancy'
params = {
  'text': 'Бортпроводник',
  'page': 0
}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

In [3]:
vacancies = []

while True:
  response = requests.get(url + route, params=params, headers=headers)
  if response.ok:
    dom = bs(response.text, 'html.parser')

    page_vacancies = dom.select('div.vacancy-serp-item')

    if not page_vacancies:
      break
    
    for page_vacancy in page_vacancies:
      vacancy_data = {
        'salary_from': None,
        'salary_to': None,
        'salary_currency': None
      }

      link_node = page_vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
      salary_node = page_vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})

      vacancy_data['title'] = link_node.text
      vacancy_data['url'] = link_node['href']
      vacancy_data['from'] = url

      if salary_node:
        salary_list = re.sub('\u202f', '', salary_node.text).split()
        vacancy_data['salary_currency'] = salary_list.pop()

        if salary_list[0] == 'от':
          vacancy_data['salary_from'] = salary_list[1]
          if 'до' in salary_list:
            vacancy_data['salary_to'] = salary_list[3]
        elif salary_list[0] == 'до':
          vacancy_data['salary_to'] = salary_list[1]
        else:
          vacancy_data['salary_from'] = salary_list[0]
          if '–' in salary_list:
            vacancy_data['salary_from'] = salary_list[0]
            vacancy_data['salary_to'] = salary_list[2]
          else:
            vacancy_data['salary_to'] = salary_list[0]

      vacancies.append(vacancy_data)

    print(f"Обработано вакансий на {params['page'] + 1} странице: {len(page_vacancies)}")
    params['page'] += 1
  else:
    break

Обработано вакансий на 1 странице: 20
Обработано вакансий на 2 странице: 20
Обработано вакансий на 3 странице: 20
Обработано вакансий на 4 странице: 20
Обработано вакансий на 5 странице: 20
Обработано вакансий на 6 странице: 4


In [4]:
data = pd.DataFrame(vacancies)
data.sample(10)

,salary_from,salary_to,salary_currency,title,url,from
42,None,None,None,Бортпроводник,https://togliatti.hh.ru/vacancy/50767094?from=...,https://hh.ru
63,42000,47000,руб.,Менеджер по персоналу,https://togliatti.hh.ru/vacancy/50156810?from=...,https://hh.ru
31,None,None,None,Бортпроводник-стажёр,https://togliatti.hh.ru/vacancy/50042619?from=...,https://hh.ru
65,45000,None,руб.,"Стюард (Красная поляна, 2200)",https://togliatti.hh.ru/vacancy/50771058?from=...,https://hh.ru
92,35000,50000,руб.,Стюард / Кухонный рабочий (банкеты и мероприятия),https://togliatti.hh.ru/vacancy/50309323?from=...,https://hh.ru
93,35000,50000,руб.,Стюард / Кухонный рабочий (банкеты и мероприятия),https://togliatti.hh.ru/vacancy/50309366?from=...,https://hh.ru
52,100000,140000,руб.,Бортпроводник,https://togliatti.hh.ru/vacancy/48069299?from=...,https://hh.ru
34,None,150000,руб.,Бортпроводник-фрилансер,https://togliatti.hh.ru/vacancy/50600307?from=...,https://hh.ru
54,None,None,None,Бортпроводник ВС Боинг-737NG/ Боинг-757/ Боинг...,https://togliatti.hh.ru/vacancy/49064313?from=...,https://hh.ru
39,1250,3270,USD,Стюардесса (стюард) морского пассажирского флота,https://togliatti.hh.ru/vacancy/50554392?from=...,https://hh.ru


In [5]:
with open('hh_vacancies.json', 'w') as f:
    json.dump(vacancies, f)

###  Задание 3

In [15]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

In [16]:
client = MongoClient('127.0.0.1', 27017)
db = client['vacansy_db']
vacancies = db.vacansy_db

In [19]:
def vacancies(value):
    if type(value) == list:
        for el in value:
            try:
                vacancies.insert_one(el)
            except TypeError:
                print('Неверный тип данных')
            except dke:
                print(f"Вакансия с id {el['_id']} уже добавлена")  
    else:
        try:
            vacancies.insert_one(value)
        except TypeError:
            print('Неверный тип данных')
        except dke:
            print("Такая вакансия уже добавлена"

SyntaxError: unexpected EOF while parsing (<ipython-input-19-3bf6ee2a5d26>, line 16)

In [20]:
def look_for_documents(wage):
    out = []

    for item in vacancies.find({'$or': [{'min_wage': {'$gt': 0}}, {'max_wage': {'$gt': 0}}]}): #нет смысла выводить то, где пустые значения в макс и мин ЗП
        if item["min_wage"] == None:
            if wage <= item['max_wage']:
                out.append(item)
        elif item["max_wage"] == None:
            if wage <= item["min_wage"]:
                out.append(item)
        else:
            if wage <= item["min_wage"]:
                out.append(item)
            elif wage <= item["max_wage"]:
                out.append(item)
    return out

In [21]:
look_for_documents(200000)

[]

In [30]:
vacancies

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'vacansy_db'), 'vacansy_db')